In [1]:
# dataapi

# -*- coding: utf-8 -*-
import httplib
import traceback
import urllib
HTTP_OK = 200
HTTP_AUTHORIZATION_ERROR = 401
class Client:
    domain = 'api.wmcloud.com'
    port = 443
    token = ''
    httpClient = None
    #设置因网络连接，重连的次数
    reconnectTimes=3
    def __init__( self ):
        self.httpClient = httplib.HTTPSConnection(self.domain, self.port, timeout=60)
    def __del__( self ):
        if self.httpClient is not None:
            self.httpClient.close()
    def encodepath(self, path):
        #转换参数的编码
        start=0
        n=len(path)
        re=''
        i=path.find('=',start)
        if(i==-1):
            return path
        while i!=-1 :
            re+=path[start:i+1]
            start=i+1
            i=path.find('&',start)
            if(i>=0):
                for j in range(start,i):
                    if(path[j]>'~'):
                        re+=urllib.quote(path[j])
                    else:
                        re+=path[j]  
                re+='&'
                start=i+1
            else:
                for j in range(start,n):
                    if(path[j]>'~'):
                        re+=urllib.quote(path[j])
                    else:
                        re+=path[j]  
                start=n
            i=path.find('=',start)
        return re
    def init(self, token):
        self.token=token
    def getData(self, path):
        result = None
        path='/data/v1'+path
        path=self.encodepath(path)
        print path
        for i in range(self.reconnectTimes):
            try:
                #set http header here
                self.httpClient.request('GET', path, headers = {"Authorization": "Bearer " + self.token})
                #make request
                response = self.httpClient.getresponse()
                #read result
                if response.status == HTTP_OK:
                    #parse json into python primitive object
                    result = response.read()
                else:
                    result = response.read()
                if(path.find('.csv?')==-1):
                    result=result.decode('utf-8').encode('GB18030')
                return response.status, result
            except Exception, e:
                #网络连接失败，关闭当前连接，再重新建立
                print i
                if self.httpClient is not None:
                    self.httpClient.close()
                self.httpClient = httplib.HTTPSConnection(self.domain, self.port, timeout=60)
        return -1,'cannot connect '+str(self.reconnectTimes)+' times'


In [2]:
#simplycode get fund nav
# -*- coding: utf-8 -*-
from dataapi import Client
if __name__ == "__main__":
    try:
        client = Client()
        client.init('ec401353e11cd728472e9027e6d900387511e6c3cf5b54d8f277b18cc45f6bcd')
        url1='/api/fund/getFundNav.json?field=&beginDate=20151228&endDate=20151231&secID=&ticker=150175'
        code, result = client.getData(url1)
        if code==200:
            '''
            print result
            secID="150175.XSHE"
            print result.find(secID)
            lens_secID=len(secID)
            print lens_secID
            str=result[50:lens_secID]
            print str
            '''
            #print "data: ",result['data']
            #-------------write to csv file---------------------------------------------
            file_object = open('C:\\pythoncoding\\programming trade\\wmcloseplate\\150175.csv', 'w')
            file_object.write(result)
            file_object.close( )
        else:
            print code
            print result
    except Exception,e:
        raise e
 

ImportError: No module named dataapi

In [ ]:
 url2='/api/subject/getThemesContent.csv?field=&themeID=&themeName=&isMain=1&themeSource='
        code, result = client.getData(url2)
        if(code==200):
            file_object = open('thefile.csv', 'w')
            file_object.write(result)
            file_object.close( )
        else:
            print code
            print result
    except Exception, e:
        #traceback.print_exc()
        raise e